# Load packages

In [ ]:
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data

In [ ]:
df = px.data.stocks(indexed=False)
df

# Wide to Long table

In [ ]:
id_vars = ["date"]
value_vars = [col for col in df.columns if col not in id_vars]
df_long = pd.melt(
    df, id_vars=id_vars, value_vars=value_vars, 
    var_name="ID", value_name="Value",
    ignore_index=True
)
df_long["date"] = pd.to_datetime(df_long["date"])
df_long

# Plot multiple time series colored by ID

In [28]:
fig = px.line(
    df_long, x="date", y="Value", color="ID",
    hover_data={"date": "|%B %d, %Y"},
    title='custom tick labels'
)

# update x-axis
fig.update_xaxes(
    dtick="M1",            # distance between two ticks
    tickformat="%b\n%Y",
    ticklabelmode="period" # Moving Tick Labels to the Middle of the Period
)

# Alternative, update layout
# fig.update_layout(
#     xaxis_dtick="M1",
#     xaxis_tickformat = '%d %B (%a)<br>%Y',
#     xaxis_ticklabelmode="period"
# )
# fig.show()
pio.write_image(fig, "../images/time_series/stock_custom_tick.png", width=1000, height=300)

<img src="../images/time_series/stock_custom_tick.png" width=1000 height=300>

# Plot using graph object

In [29]:
col_id = "ID"
col_time = "date"
col_x = "Value"

fig = go.Figure() # Create a figure

# Iterately add a trace
for ID, sub_df in df_long.groupby(by=col_id):
    fig.add_trace(
        # add graph_objects use go
        go.Scatter(
            x=sub_df[col_time],
            y=sub_df[col_x],
            mode='lines',
            name=f'{col_id} {ID}',
            hovertext=sub_df[col_time].dt.strftime('%B %d, %Y')  # Format the hover text
        )
    )

# update x-axis
fig.update_xaxes(
    dtick="M1",            # distance between two ticks
    tickformat="%b\n%Y",
    ticklabelmode="period"  # Moving Tick Labels to the Middle of the Period
)

# update title
fig.update_layout(
    title='Custom Tick Labels'
)

# fig.show()
# pio.write_image(fig, "../images/time_series/stock_custom_tick_mode_period.png", width=1000, height=300)

<img src="../images/time_series/stock_custom_tick_mode_period.png" width=1000 height=300>

# Compute simple return

$$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}} = \frac{P_t}{P_{t-1}} - 1
$$

In [ ]:
df_long['R_t'] = df_long.groupby(by=col_id, as_index=False)[col_x]\
    .transform(lambda x: (x/x.shift(1) - 1))
df_long

# Dual axis plot with grouped legend

Can only be created using graph objects.

In [ ]:
def hex_to_rgb(hex_color: str) -> str:
    """change hex color code to rgb color code

    Example
    ---------
    >>> hex_color = '#FFAABB'
    >>> hex_to_rgb(hex_color)
    >>> (255, 170, 187)
    """
    hex_color = hex_color.lstrip('#')  # Remove the '#' character if it's present
    return tuple(int(hex_color[i:i + 2], 16) for i in (0, 2, 4))

In [30]:
color_palette = px.colors.qualitative.Alphabet

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Iterately add a trace
for i, (ID, sub_df) in enumerate(df_long.groupby(by=col_id)):
    # note some of color code are hex, some are rgb
    color = str(hex_to_rgb(color_palette[i]))
    color_P = f"rgba{color[:-1]}, 1)"      # opacity/alpha = 1
    color_R = f"rgba{color[:-1]}, 0.7)"    # opacity/alpha = 0.7

    fig.add_trace(go.Scatter(
        x=sub_df[col_time],
        y=sub_df[col_x],
        mode='lines',
        name='P_t',
        legendgroup=ID,                    # Specify the legend group
        legendgrouptitle_text=f"ID: {ID}", # Specify the legend group title
        line=dict(color=color_P, width=2),
        yaxis='y'  # Specify the primary y-axis
    ))

    fig.add_trace(go.Scatter(
        x=sub_df[col_time],
        y=sub_df['R_t'],
        mode='lines',
        name=f'R_t',
        legendgroup=ID,                    # Specify the legend group
        line=dict(color=color_R, width=2),
        yaxis='y2'  # Specify the secondary y-axis
    ))

color = str(hex_to_rgb(color_palette[i+5]))
color_P = f"rgba{color[:-1]}, 1)"      
color_R = f"rgba{color[:-1]}, 0.7)"    

fig.update_layout(
    title=dict(
        text="<b>Stocks P_t and R_t<b>", # <b> to set bold
        font=dict(
            size=20,
            family="Times New Roman",
            color="blue",
        ),
    ),
    yaxis=dict(
        title="P_t",
        titlefont=dict(
            color=color_P
        ),
        tickfont=dict(
            color=color_P
        ),
        anchor="x",
    ),
    yaxis2=dict(
        title="R_t",
        titlefont=dict(
            color=color_R
        ),
        tickfont=dict(
            color=color_R
        ),
        tickmode="sync", # sync the axis
        anchor="x",
        overlaying="y",  # overlap with y
        side="right",
    ),
)

# fig.show()
# pio.write_image(fig, "../images/time_series/grouped_legend_dual_axis.png", width=1500, height=400)

<img src="../images/time_series/grouped_legend_dual_axis.png" width=1500 height=400>

# Colors

## Qualitative

In [ ]:
import plotly.express as px

fig = px.colors.qualitative.swatches()
fig.show()

print(px.colors.qualitative.Plotly)

## Sequential

In [ ]:
import plotly.express as px

fig = px.colors.sequential.swatches_continuous()
fig.show()

print(px.colors.sequential.Viridis)

## Diverging

In [ ]:
import plotly.express as px

fig = px.colors.diverging.swatches_continuous()
fig.show()

# Reference

- [Time Series and Date Axes in Python](https://plotly.com/python/time-series/)
- [Multiple Axes in Python](https://plotly.com/python/multiple-axes/)
- [Discrete Colors in Python](https://plotly.com/python/discrete-color/)
- [Built-in Continuous Color Scales in Python](https://plotly.com/python/builtin-colorscales/)